Challenge Intermedio: Modelado con Spark ML
1.	Introducción:

Este challenge lleva a los estudiantes un paso más allá en el uso de PySpark, implementando Spark ML para realizar un modelo predictivo basado en una gran base de datos.
2.	Objetivo general:

Aplicar algoritmos de machine learning en grandes bases de datos utilizando Spark ML.
3.	Objetivo específico:

Descargar una base de datos desde UCI Machine Learning Repository, limpiarla, y entrenar un modelo predictivo de clasificación utilizando Spark ML.

4.	Base de datos:

Base de datos: Wine Quality Dataset
Plataforma: UCI Machine Learning Repository
Instrucciones: Descargar y cargar los datos en Databricks para crear el modelo.
5.	API y herramientas:

Usar Spark ML para entrenar un modelo de regresión logística que prediga la calidad del vino. 

6.	Repositorio GitHub:

Subir el código del modelo y un informe de los resultados a GitHub.


In [1]:
# Importar librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 1. Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Wine Quality Prediction") \
    .getOrCreate()

# 2. Cargar el conjunto de datos
# Descargar el dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
file_path = "/dbfs/tmp/winequality-red.csv"  # Ruta en Databricks
data = spark.read.csv(file_path, header=True, inferSchema=True, sep=";")

# Mostrar la estructura de los datos
data.printSchema()

# 3. Preprocesamiento de datos
# Revisar valores nulos
data.select([col(c).isNull().alias(c) for c in data.columns]).show()

# Convertir la columna "quality" en una variable categórica binaria (ejemplo: alta calidad = 1, baja calidad = 0)
data = data.withColumn("label", when(col("quality") >= 7, 1).otherwise(0))
data = data.drop("quality")  # Eliminar la columna original

# Verificar el balance de clases
data.groupBy("label").count().show()

# 4. Crear características para el modelo
feature_columns = [col for col in data.columns if col != "label"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features_raw")
data = assembler.transform(data)

# Escalar las características
scaler = StandardScaler(inputCol="features_raw", outputCol="features", withStd=True, withMean=False)
scaler_model = scaler.fit(data)
data = scaler_model.transform(data)

# Dividir en conjuntos de entrenamiento y prueba
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# 5. Crear y entrenar el modelo
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)
model = lr.fit(train_data)

# 6. Evaluar el modelo
predictions = model.transform(test_data)
predictions.select("label", "prediction", "probability").show(10)

# Evaluar la precisión
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")

# Guardar el modelo en Databricks (opcional)
model_path = "/dbfs/tmp/wine_quality_model"
model.write().overwrite().save(model_path)

# Detener la sesión de Spark
spark.stop()


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/dbfs/tmp/winequality-red.csv.